<a href="https://colab.research.google.com/github/TienLungSun/Intelligent-Robot/blob/main/LearnPPO-AC/6.1.%20A2C%20use%20Tensorboard%20(MLAgent_10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import numpy as np
import matplotlib.pyplot as plt
from mlagents_envs.environment import UnityEnvironment

In [ ]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cuda GeForce GTX 1660 SUPER


### Training parameters (in Captical letters)

In [ ]:
N_STATES  = 8
N_ACTIONS =2

N_AGENTS = 3   #My Unity scene has three training environment

BATCH_SIZE = 64  
BUFFER_SIZE = 12000
LEARNING_RATE = 0.0003
BETA = 0.001
EPSILON = 0.2
LAMBD = 0.99
N_EPOCH = 3

GAMMA = 0.99

MAX_STEPS = 5000 #50000
TIME_HORIZON = 50 #1000 

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)

In [ ]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(N_STATES, 128),
            nn.LayerNorm(128),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.Linear(128, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(N_STATES, 128),
            nn.LayerNorm(128),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.Linear(128, N_ACTIONS)
        )
        self.log_std = nn.Parameter(torch.ones(1, N_ACTIONS) * 0.0)
        self.apply(init_weights)
    
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [ ]:
net = Net().to(device)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE )

### function to interacts with Unity NoSteps

In [ ]:
def collect_training_data (NoAgents, NoSteps, NoStates, NoActions, env, net):
    # collect (s,a,r,s1) from 20 good interactions with Unity
    a = torch.FloatTensor([[0]]*NoAgents) #create NoAgents by 1 tensor
    b = torch.FloatTensor([[0]*NoActions]*NoAgents) 
    c = torch.FloatTensor([[0]*NoStates]*NoAgents) 
    values =rewards = masks = [a]*NoSteps
    log_probs = actions = [b]*NoSteps
    states =[c]*NoSteps
    next_state = c

    step = 0  #index 0
    DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
    while(step < NoSteps): #try to run 20 good steps
        if(len(DecisionSteps) == 0): #we have no decision agents
            env.reset() 
            DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
            #continue next while loop without increase step
        else:  # we have decision agents
            lst = list(DecisionSteps.agent_id) + list(TerminalSteps.agent_id)
            if(len(list(set(lst))) != NoAgents): 
                #if this step misses some agents, then only interacts with Unity 
                #but do not collect data
                s = DecisionSteps.obs[0]  
                s = torch.FloatTensor(s).to(device)    
                dist, value = net(s)
                a = dist.sample() 
                env.set_actions(behaviorName, a.cpu().detach().numpy() )   
                env.step()
                DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
                #continue next while loop without increase step
            else: 
                #this step includes all agents, collect (s, a, r, s1)
                s = DecisionSteps.obs[0]  
                s = torch.FloatTensor(s).to(device)         
                dist, value = net(s)
                a = dist.sample()
                log_prob = dist.log_prob(a)
                for idx in range(len(DecisionSteps)):
                    #find decision agents and record their state, value and actions
                    AgentID = DecisionSteps.agent_id[idx]
                    states[step][AgentID]=s[idx]
                    values[step][AgentID]=value[idx]
                    actions[step][AgentID]=a[idx]
                    log_probs[step][AgentID]=log_prob[idx]
                env.set_actions(behaviorName, a.cpu().detach().numpy() )   
                env.step()
                DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
                #if next step misses some agents, then do not update step
                lst = list(DecisionSteps.agent_id) + list(TerminalSteps.agent_id)
                if(len(list(set(lst))) != NoAgents):
                    continue #skip this step, run next while loop without increase step index
                else:
                    #collect reward of this action from next decision and terminal steps
                    if(len(TerminalSteps) >0):
                        #if next step has terminal agents, then collect terminal agents first
                        r = TerminalSteps.reward
                        r = torch.FloatTensor(r).unsqueeze(1)
                        s = TerminalSteps.obs[0]  
                        s = torch.FloatTensor(s).to(device) 
                        for idx in range(len(TerminalSteps)):
                            AgentID = TerminalSteps.agent_id[idx]
                            rewards[step][AgentID]=r[idx]
                            masks[step][AgentID]= 0
                            next_state[AgentID]=s[idx]
                    #collect from decision steps
                    r = DecisionSteps.reward 
                    r = torch.FloatTensor(r).unsqueeze(1)
                    s = DecisionSteps.obs[0]  
                    s = torch.FloatTensor(s).to(device) 
                    for idx in range(len(DecisionSteps)):
                        AgentID = DecisionSteps.agent_id[idx] 
                        if(rewards[step][AgentID] == None): #this agent is not in terminal step
                            rewards[step][AgentID]= r[idx]
                            masks[step][AgentID]= 1
                            next_state[AgentID]=s[idx]
                    step = step + 1 #increase step and run next while
    return  states, actions, log_probs, values, rewards, masks, next_state

### GAE

In [ ]:
def compute_gae(NoSteps, next_value, rewards, masks, values, gamma, tau):
    values = values + [next_value.cpu()]
    gae = 0
    returns = []
    for step in reversed(range(NoSteps)):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

### PPO optimization

In [ ]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
    batch_size = states.size(0)
    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantages[rand_ids, :]

In [ ]:
def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    print("epoch:")
    for epoch in range(ppo_epochs):
        print(epoch, end = ", ")
        for batch_state, batch_action, batch_old_log_probs, batch_return, batch_advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = net(batch_state.to(device))
            critic_loss = (batch_return.to(device) - value).pow(2).mean()
            entropy = dist.entropy().mean()
            batch_action = dist.sample()
            batch_new_log_probs = dist.log_prob(batch_action)
            ratio = (batch_new_log_probs - batch_old_log_probs.to(device)).exp()
            surr1 = ratio * batch_advantage.to(device)
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * batch_advantage.to(device)
            actor_loss  = - torch.min(surr1, surr2).mean()
            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return float(critic_loss), float(actor_loss)

### Start Tensorboard server

In [ ]:
# current directory of this ipython notebook 
!echo The current directory is %CD%

The current directory is C:\Users\ADMIN\Google 雲端硬碟\0. 教學 IE 351, 562 VR1 Unity 3D


In [ ]:
# Run python terminal window
# Change directory to this ipython notebook directory
# tensorboard --logdir=runs

In [ ]:
# Web browser:  localhost:6006 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# run PPO optimization multiple iterations

In [ ]:
BATCH_SIZE = 64  
BUFFER_SIZE = 12000
N_EPOCH = 3

BETA = 0.001
EPSILON = 0.2
LAMBD = 0.99
GAMMA = 0.99

MAX_STEPS = 500 #50000
TIME_HORIZON = 50 #1000 

In [ ]:
env = UnityEnvironment(file_name= None, base_port=5004)

In [ ]:
env.reset()
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]
print(behaviorName)

3DBall?team=0


In [ ]:
frame_idx  = 0 

while (frame_idx < MAX_STEPS):
    print("\nframe idx = ", frame_idx)
    print("Interacts with Unity to collect training data")
    states, actions, log_probs, values, rewards, masks, next_state = collect_training_data (N_AGENTS, TIME_HORIZON, N_STATES, N_ACTIONS, env, net)
    _, next_value = net(next_state.to(device)) 
    
    print("Compute GAE of these training data set")
    returns = compute_gae(TIME_HORIZON, next_value, rewards, masks, values, GAMMA, LAMBD)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states) 
    actions   = torch.cat(actions)
    advantages = returns - values
    
    print("Optimize NN with PPO")
    critic_loss, actor_loss = ppo_update(N_EPOCH, BATCH_SIZE, states, actions, log_probs, returns, advantages, clip_param=0.2)    
    writer.add_scalar("Actor loss", actor_loss, frame_idx)
    writer.add_scalar("Critic loss", critic_loss, frame_idx)

    frame_idx += TIME_HORIZON


frame idx =  0
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  50
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  100
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  150
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  200
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  250
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  300
Interacts with Unity to collect training data
Compute GAE of these training data set
Optimize NN with PPO
epoch:
0, 1, 2, 
frame idx =  350
Inter

In [ ]:
env.close()